In [17]:
%pip install -q pypdf
%pip install -q faiss-cpu
%pip install -q rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import bs4
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.document_loaders import PyPDFLoader

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = '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'

In [2]:


query = "What does television fueled inventors to invent?"
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def load_from_url(url):
    loader = WebBaseLoader(
        web_paths=(url,),
    )
    return loader.load_and_split()


def load_from_pdf_documents(file_path):
    loader = PyPDFLoader(file_path, extract_images=True)
    pages = loader.load()
    return pages

In [36]:
docs = load_from_pdf_documents("./documents/pdf/Syllabus.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# retrieve and generate
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 6})

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def load_from_url(url):
    loader = WebBaseLoader(
        web_paths=(url,),
    )
    return loader.load_and_split()


def load_from_pdf_documents(file_path):
    loader = PyPDFLoader(file_path, extract_images=True)
    pages = loader.load()
    return pages


def rag_url_agent(query: str):

    memory = SqliteSaver.from_conn_string(":memory:")
    llm = ChatOpenAI(model="gpt4-o", temperature=0)

    ### Construct retriever ###
    # docs = load_from_url('https://artsandculture.google.com/story/EQURMn3lzjV8JQ')
    docs = load_from_pdf_documents("./documents/pdf/Syllabus.pdf")
    print(f'Successfully loaded content from website: {docs[0].page_content[:20]}...')

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    ### Build retriever tool ###
    # tool = create_retriever_tool(
    #     retriever,
    #     "blog_post_retriever",
    #     "Searches and returns excerpts from the Autonomous Agents blog post.",
    # )
    
    tool = create_retriever_tool(
    retriever,
    "syllabus_content_retriever",
    "queries the syllabus and for relevant information",
    )
    tools = [tool]

    agent_executor = create_react_agent(llm, tools, checkpointer=memory)

    config = {"configurable": {"thread_id": "abc123"}}

    response = agent_executor.invoke(
        {"messages": [HumanMessage(content=query)]}, config=config
    )
    print(response['messages'])
    print('-------')
    print(StrOutputParser().invoke(response['messages'][-1]))
    # StrOutputParser().batch(res['messages'])[-1]



In [4]:
# query = "What does television fueled inventors to invent? "
query = "Who are the instructors and what are their emails? "
rag_url_agent(query)

Successfully loaded content from website: SyllabusJOUR1501Sect...


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 530f3652-79ba-4de5-9a06-2a74d4996115, but expected {'tool'} run.")


[HumanMessage(content='Who are the instructors and what are their emails? ', id='d43579f1-8958-46f0-b51e-5fc26c4f3e44'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_M2ebNpzmBwvGRYIERAPcI3jm', 'function': {'arguments': '{"query":"instructors"}', 'name': 'syllabus_content_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 72, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1b0df749-a678-4ba8-bf11-00c37e5ecc43-0', tool_calls=[{'name': 'syllabus_content_retriever', 'args': {'query': 'instructors'}, 'id': 'call_M2ebNpzmBwvGRYIERAPcI3jm'}], usage_metadata={'input_tokens': 72, 'output_tokens': 20, 'total_tokens': 92}), ToolMessage(content='SyllabusJOUR1501Section001:DigitalGamesandSocietySpring2024|3credits|MeetsArtsandHumanitiesCore&TechnologyandSocietyThemeLiberalEducationrequirementsFullyOnline|AsynchronousInstructo

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("Who are the instructor?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

20: outside
the
classroom.
Students
should
expect
to
spend
about
9
hours
in
learning
eﬀorts
per
week
(inside
and
outside
of
class)
to
satisfactorily
complete
this
course.
S t a t e m e n t
o f
I n s t r u c t i o n a l
T i m e
All
course
content
is
available
to
students
on
the
ﬁrst
day
of
the
course,
wi
0: Syllabus
JOUR
1501
Section
001:
Digital
Games
and
Society
Spring
2024
|
3
credits
|
Meets
Arts
and
Humanities
Core
&
Technology
and
Society
Theme
Liberal
Education
requirements
Fully
Online
|
Asynchronous
I n s t r u c t o r s
Dr.
Colin
Agur,
Associate
Professor
Email:
cpagur@umn.edu
We
are
also
ple
